In [192]:
import pandas as pd
from numpy import nan
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# Home Data pipeline

See [readme](../README.md) 2do section for next steps

In [193]:
# load data

global training___data, validation_data, training___price, validation_price, full_data, full_price, test_data
def load_data():
    global training___data, validation_data, training___price, validation_price, full_data, full_price, test_data
    test_data = pd.read_csv("./test.csv")
    full_data = pd.read_csv("./train.csv")
    full_price = full_data.SalePrice
    training___data, validation_data, training___price, validation_price = train_test_split(full_data, full_price)
    training___data = training___data.drop("SalePrice", axis=1)
    validation_data = validation_data.drop("SalePrice", axis=1)
    full_data = full_data.drop("SalePrice", axis=1)

load_data()
training___data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
528,529,30,RL,58.0,9098,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,4,7,1920,2002,Gable,CompShg,Wd Sdng,Wd Sdng,NaN,0.0,TA,TA,BrkTil,TA,TA,Mn,ALQ,348,Unf,0,180,528,GasA,Ex,Y,SBrkr,605,0,0,605,1,0,1,0,2,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,0,0,144,0,0,0,NaN,NaN,NaN,0,7,2007,WD,Normal
1240,1241,60,RL,65.0,8158,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,252.0,Gd,TA,PConc,Gd,TA,No,GLQ,550,Unf,0,334,884,GasA,Ex,Y,SBrkr,884,884,0,1768,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,543,TA,TA,Y,0,63,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal
848,849,50,RL,75.0,45600,Pave,NaN,IR2,Bnk,AllPub,Inside,Gtl,ClearCr,Norm,Norm,1Fam,1.5Fin,6,8,1908,1997,Gable,CompShg,Wd Sdng,Wd Sdng,NaN,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,907,907,GasA,TA,Y,SBrkr,1307,1051,0,2358,0,0,3,0,5,1,TA,10,Typ,1,Gd,Detchd,1908.0,Unf,2,360,Fa,TA,Y,486,40,0,0,175,0,NaN,NaN,NaN,0,9,2008,WD,Normal
1002,1003,20,RL,75.0,11957,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Somerst,RRAn,Norm,1Fam,1Story,8,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,BrkFace,53.0,Gd,TA,PConc,Gd,TA,No,GLQ,24,Unf,0,1550,1574,GasA,Ex,Y,SBrkr,1574,0,0,1574,0,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,3,824,TA,TA,Y,144,104,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal
534,535,60,RL,74.0,9056,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,8,5,2004,2004,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,Gd,TA,PConc,Ex,Gd,Av,Unf,0,Unf,0,707,707,GasA,Ex,Y,SBrkr,707,707,0,1414,0,0,2,1,3,1,Gd,6,Typ,1,Gd,Attchd,2004.0,Fin,2,403,TA,TA,Y,100,35,0,0,0,0,NaN,NaN,NaN,0,10,2006,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,574,80,RL,76.0,9967,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,SLvl,7,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,384,384,GasA,Ex,Y,SBrkr,774,656,0,1430,0,0,2,1,3,1,TA,8,Typ,1,TA,BuiltIn,2000.0,RFn,2,400,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,12,2007,WD,Normal
101,102,60,RL,77.0,9206,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,2Story,6,5,1985,1985,Gable,CompShg,HdBoard,HdBoard,BrkFace,336.0,Gd,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,741,741,GasA,TA,Y,SBrkr,977,755,0,1732,0,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,1985.0,Fin,2,476,TA,TA,Y,192,46,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
688,689,20,RL,60.0,8089,Pave,NaN,Reg,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,1Fam,1Story,8,6,2007,2007,Gable,CompShg,MetalSd,MetalSd,BrkFace,0.0,Gd,TA,PConc,Gd,TA,Av,GLQ,945,Unf,0,474,1419,GasA,Ex,Y,SBrkr,1419,0,0,1419,1,0,2,0,2,1,Gd,7,Typ,1,Gd,Attchd,2007.0,RFn,2,567,TA,TA,Y,140,0,0,0,0,0,NaN,NaN,NaN,0,10,2007,New,Partial
1224,1225,60,RL,60.0,15384,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,RRAn,Norm,1Fam,2Story,7,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,Gd,TA,PConc,Gd,TA,Av,GLQ,724,Unf,0,64,788,GasA,Ex,Y,SBrkr,788,702,0,1490,1,0,2,1,3,1,Gd,8,Typ,1,Gd,Attchd,2004.0,F

In [194]:
# define possible column transformations

is_nan = lambda x: x != x
equals = lambda a, b: (is_nan(a) and is_nan(b)) or a == b
all_unique_values = lambda column_name: set(training___data[column_name].unique()) | set(validation_data[column_name].unique())

def drop(column_name, data_set, *_):
    data_set.drop(column_name, axis=1, inplace=True)
    return data_set

def drop_row(column_name, data_set, condition):
    return data_set[~data_set[column_name].apply(condition)]

def one_hot(column_name, data_set, options):
    keys = options if options else data_set[column_name].unique()
    for key in keys:
        data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
    return drop(column_name, data_set)

def apply(column_name, data_set, function):
    data_set[column_name] = data_set[column_name].apply(function)
    return data_set

def re_map(column_name, data_set, mapping):
    data_set[column_name] = data_set[column_name].apply(lambda x: mapping[x])
    return data_set

def MSZoning(column_name, data_set, *_):
    options = dict(
        # A="Agriculture",
        C="Commercial",
        FV="Floating Village Residential",
        # I="Industrial",
        RH="Residential High Density",
        RL="Residential Low Density",
        # RP="Residential Low Density Park",
        RM="Residential Medium Density",
    )
    for key, name in options.items():
        if key == "C":
            continue
        data_set[f"zoned_{name}"] = data_set[column_name].apply(lambda x: 1 if x in [key, "C"] else 0)
    return drop(column_name, data_set)

def masonry(column_name, data_set, *_):
    options = dict(
        BrkCmn="Brick Common",
        BrkFace="Brick Face",
        CBlock="Cinder Block",
        Stone="Stone",
    )
    for key in options:
        data_set[f"masonry {options[key]}"] = 0
        for id in range(len(data_set)):
            if data_set[column_name].iloc[id] == key:
                data_set[f"masonry {options[key]}"].iat[id] = data_set["MasVnrArea"].iloc[id]
    
    data_set.drop(["MasVnrType", "MasVnrArea"], axis=1, inplace=True)
    return data_set

def basement(column_name, data_set, *_):
    options = dict(
        GLQ="Good Living Quarters",
        ALQ="Average Living Quarters",
        BLQ="Below Average Living Quarters",
        Rec="Average Rec Room",
        LwQ="Low Quality",
        Unf="Unfinished",
    )
    for key in options:
        data_set[f"basement {options[key]} sqft"] = 0
        for id in range(len(data_set)):
            if data_set["BsmtFinType1"].iloc[id] == key:
                data_set[f"basement {options[key]} sqft"].iat[id] += data_set["BsmtFinSF1"].iloc[id]
            if data_set["BsmtFinType2"].iloc[id] == key:
                data_set[f"basement {options[key]} sqft"].iat[id] += data_set["BsmtFinSF2"].iloc[id]
    
    data_set.drop(["BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2"], axis=1, inplace=True)
    return data_set

def basement_unfinished(column_name, data_set, *_):
    unf1 = "basement Unfinished sqft"
    unf2 = "BsmtUnfSF"
    data_set[unf1] = data_set.apply(lambda x: x[unf1] + x[unf2], axis=1)
    
    data_set.drop(["BsmtUnfSF"], axis=1, inplace=True)
    return data_set

def garage_year(column_name, data_set, *_):
    data_set["GarageYrBlt"] = data_set.apply(lambda x: x["YearRemodAdd"] if is_nan(x["GarageYrBlt"]) else x["GarageYrBlt"], axis=1)
    return data_set

def NA_to_zero(column_name, data_set, *_):
    data_set[column_name] = data_set[column_name].apply(lambda x: 0 if is_nan(x) else x)
    return data_set

import inflation
def sale_price(column_name, data_set, *_):
    if column_name in data_set:
        for id in range(len(data_set)):
            data_set[column_name].iat[id] = inflation.adjust(
                price = data_set[column_name].iloc[id],
                year = data_set["YrSold"].iloc[id],
                month = data_set["MoSold"].iloc[id],
            )
    data_set.drop(["MoSold", "YrSold"], axis=1, inplace=True)
    return data_set

In [195]:
# specify each column transformation

transformations = {
    "MSSubClass": (drop, None),
    "MSZoning": (MSZoning, None),
    "LotFrontage": (apply, lambda x: 0 if is_nan(x) else x),
    "Street": (re_map, {"Pave": 1, "Grvl": 0}),
    "Alley": (drop, None),
    "LotShape": (re_map, {"Reg": 0, "IR1": 1, "IR2": 2, "IR3": 3}),
    "LandContour": (re_map, {"Lvl": 1, "Bnk": 3, "HLS": 2, "Low": 0}),
    "Utilities": (drop, None),
    "LotConfig": (one_hot, all_unique_values("LotConfig")),
    "LandSlope": (re_map, {"Gtl": 0, "Mod": 1, "Sev": 2}),
    "Neighborhood": (one_hot, all_unique_values("Neighborhood")),
    "Condition1": (drop, None),
    "Condition2": (drop, None),
    "BldgType": (one_hot, all_unique_values("BldgType")),
    "HouseStyle": (one_hot, all_unique_values("HouseStyle")),
    "YearBuilt": (drop, None),
    "RoofStyle": (one_hot, all_unique_values("RoofStyle")),
    "RoofMatl": (one_hot, all_unique_values("RoofMatl")),
    "Exterior1st": (one_hot, all_unique_values("Exterior1st")),
    "Exterior2nd": (drop, None),
    "MasVnrType": (masonry, None),
    "ExterQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "ExterCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "Foundation": (one_hot, all_unique_values("Foundation")),
    "BsmtQual": (re_map, {"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0, nan: 0}),
    # "BsmtCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, "NA": 2, nan: 2}),
    "BsmtCond": (drop, None),
    "BsmtExposure": (re_map, {"Gd": 3, "Av": 2, "Mn": 1, "No": 0, "NA": 0, nan: 0}),
    "TotalBsmtSF": (basement, None),
    "BsmtUnfSF": (basement_unfinished, None),
    "Heating": (one_hot, all_unique_values("Heating")),
    "HeatingQC": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "CentralAir": (re_map, {"Y": 1, "N": 0}),
    "Electrical": (re_map, {"SBrkr": 4, "FuseA": 3, "FuseF": 2, "FuseP": 1, "Mix": 0, nan: 0}),
    "BsmtFullBath": (NA_to_zero, None),
    "BsmtHalfBath": (NA_to_zero, None),
    "KitchenQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, nan: 0}),
    "Functional": (re_map, {"Typ": 7, "Min1": 6, "Min2": 5, "Mod": 4, "Maj1": 3, "Maj2": 2, "Sev": 1, "Sal": 0, nan: 7}),
    "FireplaceQu": (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, nan: 3}),
    "GarageType": (one_hot, all_unique_values("GarageType")),
    "GarageYrBlt": (garage_year, None),
    "GarageFinish": (re_map, {"Fin": 2, "RFn": 1, "Unf": 0, "NA": 1, nan: 1}),
    "GarageCars": (NA_to_zero, None),
    "GarageQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, "NA": 2, nan: 2}),
    "GarageCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, "NA": 2, nan: 2}),
    "PavedDrive": (re_map, {"Y": 2, "P": 1, "N": 0}),
    "PoolQC": (re_map, {"Ex": 5, "Gd": 4, "TA": 2, "Fa": 1, nan: 3}),
    "Fence": (one_hot, all_unique_values("Fence")),
    "MiscFeature": (drop, None),
    "SaleType": (one_hot, all_unique_values("SaleType")),
    "SaleCondition": (one_hot, all_unique_values("SaleCondition")),
    "SalePrice": (sale_price, None),
}

In [196]:
# apply column transformations to data
load_data()
for column_name, (transform, option) in transformations.items():
    print("transforming", column_name)
    training___data = transform(column_name, training___data, option)
    validation_data = transform(column_name, validation_data, option)

validation_data

transforming MSSubClass
transforming MSZoning
transforming LotFrontage
transforming Street
transforming Alley
transforming LotShape
transforming LandContour
transforming Utilities
transforming LotConfig
transforming LandSlope
transforming Neighborhood
transforming Condition1
transforming Condition2
transforming BldgType
transforming HouseStyle
transforming YearBuilt
transforming RoofStyle
transforming RoofMatl
transforming Exterior1st
transforming Exterior2nd
transforming MasVnrType
transforming ExterQual
transforming ExterCond
transforming Foundation
transforming BsmtQual
transforming BsmtCond
transforming BsmtExposure
transforming TotalBsmtSF
transforming BsmtUnfSF
transforming Heating
transforming HeatingQC
transforming CentralAir
transforming Electrical
transforming BsmtFullBath
transforming BsmtHalfBath
transforming KitchenQual
transforming Functional
transforming FireplaceQu
transforming GarageType
transforming GarageYrBlt
transforming GarageFinish
transforming GarageCars
transfo

,Id,LotFrontage,LotArea,Street,LotShape,LandContour,LandSlope,OverallQual,OverallCond,YearRemodAdd,ExterQual,ExterCond,BsmtQual,BsmtExposure,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LotConfig_FR2,LotConfig_CulDSac,LotConfig_Inside,LotConfig_FR3,LotConfig_Corner,Neighborhood_Mitchel,Neighborhood_Timber,Neighborhood_SawyerW,Neighborhood_Edwards,Neighborhood_NridgHt,Neighborhood_IDOTRR,Neighborhood_BrkSide,Neighborhood_Gilbert,Neighborhood_Sawyer,Neighborhood_CollgCr,Neighborhood_OldTown,Neighborhood_NoRidge,Neighborhood_NPkVill,Neighborhood_NAmes,Neighborhood_ClearCr,Neighborhood_NWAmes,Neighborhood_Veenker,Neighborhood_Somerst,Neighborhood_BrDale,Neighborhood_MeadowV,Neighborhood_Crawfor,Neighborhood_StoneBr,Neighborhood_SWISU,Neighborhood_Blueste,Neighborhood_Blmngtn,BldgType_TwnhsE,BldgType_1Fam,BldgType_Twnhs,BldgType_Duplex,BldgType_2fmCon,HouseStyle_2.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_1.5Fin,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Mansard,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Shed,RoofStyle_Flat,RoofMatl_WdShngl,RoofMatl_WdShake,RoofMatl_ClyTile,RoofMatl_Membran,RoofMatl_Tar&Grv,RoofMatl_Roll,RoofMatl_CompShg,RoofMatl_Metal,Exterior1st_MetalSd,Exterior1st_BrkComm,Exterior1st_AsbShng,Exterior1st_Wd Sdng,Exterior1st_ImStucc,Exterior1st_Stucco,Exterior1st_Plywood,Exterior1st_CBlock,Exterior1st_WdShing,Exterior1st_Stone,Exterior1st_CemntBd,Exterior1st_AsphShn,Exterior1st_VinylSd,Exterior1st_BrkFace,Exterior1st_HdBoard,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_Stone,Foundation_PConc,Foundation_Wood,Foundation_Slab,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,Heating_GasA,Heating_OthW,Heating_GasW,Heating_Wall,Heating_Grav,Heating_Floor,GarageType_BuiltIn,GarageType_Detchd,GarageType_Basment,GarageType_2Types,GarageType_Attchd,GarageType_nan,GarageType_CarPort,Fence_MnWw,Fence_MnPrv,Fence_GdWo,Fence_GdPrv,Fence_nan,SaleType_WD,SaleType_Oth,SaleType_COD,SaleType_ConLD,SaleType_ConLI,SaleType_CWD,SaleType_ConLw,SaleType_New,SaleType_Con,SaleCondition_AdjLand,SaleCondition_Partial,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Alloca
198,199,92.0,5520,1,0,1,0,6,6,1950,2,2,85,0,755,4,1,4,929,929,371,2229,0,0,1,0,5,1,2,8,7,0,3,1950.0,1,0,0,2,2,2,0,198,30,0,0,0,3,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,755,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
487,488,70.0,12243,1,1,1,0,5,6,1971,2,2,95,2,1484,3,1,4,1484,0,0,1484,0,0,2,0,3,1,2,7,7,1,3,1971.0,0,2,487,2,2,2,224,0,0,0,180,0,3,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,998,0,0,0,486,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
358,359,92.0,6930,1,1,1,0,5,4,1958,2,2,85,2,1062,4,1,2,1352,0,0,1352,0,1,1,0,3,1,3,6,5,0,3,1958.0,0,1,288,2,2,2,168,0,294,0,0,0,3,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,120,0,0,0,1,0,0,0,0,0,0,300,294,0,468,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0

In [197]:
training___data.describe()
validation_data.describe()

,Id,LotFrontage,LotArea,Street,LotShape,LandContour,LandSlope,OverallQual,OverallCond,YearRemodAdd,ExterQual,ExterCond,BsmtQual,BsmtExposure,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LotConfig_FR2,LotConfig_CulDSac,LotConfig_Inside,LotConfig_FR3,LotConfig_Corner,Neighborhood_Mitchel,Neighborhood_Timber,Neighborhood_SawyerW,Neighborhood_Edwards,Neighborhood_NridgHt,Neighborhood_IDOTRR,Neighborhood_BrkSide,Neighborhood_Gilbert,Neighborhood_Sawyer,Neighborhood_CollgCr,Neighborhood_OldTown,Neighborhood_NoRidge,Neighborhood_NPkVill,Neighborhood_NAmes,Neighborhood_ClearCr,Neighborhood_NWAmes,Neighborhood_Veenker,Neighborhood_Somerst,Neighborhood_BrDale,Neighborhood_MeadowV,Neighborhood_Crawfor,Neighborhood_StoneBr,Neighborhood_SWISU,Neighborhood_Blueste,Neighborhood_Blmngtn,BldgType_TwnhsE,BldgType_1Fam,BldgType_Twnhs,BldgType_Duplex,BldgType_2fmCon,HouseStyle_2.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_1.5Fin,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Mansard,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Shed,RoofStyle_Flat,RoofMatl_WdShngl,RoofMatl_WdShake,RoofMatl_ClyTile,RoofMatl_Membran,RoofMatl_Tar&Grv,RoofMatl_Roll,RoofMatl_CompShg,RoofMatl_Metal,Exterior1st_MetalSd,Exterior1st_BrkComm,Exterior1st_AsbShng,Exterior1st_Wd Sdng,Exterior1st_ImStucc,Exterior1st_Stucco,Exterior1st_Plywood,Exterior1st_CBlock,Exterior1st_WdShing,Exterior1st_Stone,Exterior1st_CemntBd,Exterior1st_AsphShn,Exterior1st_VinylSd,Exterior1st_BrkFace,Exterior1st_HdBoard,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_Stone,Foundation_PConc,Foundation_Wood,Foundation_Slab,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,Heating_GasA,Heating_OthW,Heating_GasW,Heating_Wall,Heating_Grav,Heating_Floor,GarageType_BuiltIn,GarageType_Detchd,GarageType_Basment,GarageType_2Types,GarageType_Attchd,GarageType_nan,GarageType_CarPort,Fence_MnWw,Fence_MnPrv,Fence_GdWo,Fence_GdPrv,Fence_nan,SaleType_WD,SaleType_Oth,SaleType_COD,SaleType_ConLD,SaleType_ConLI,SaleType_CWD,SaleType_ConLw,SaleType_New,SaleType_Con,SaleCondition_AdjLand,SaleCondition_Partial,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Alloca
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.00000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.0,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.0,365.000000,365.000000,365.000000

# Fit & compare

In [198]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

errors = []
for n in range(10):
    load_data()
    for column_name, (transform, option) in transformations.items():
        training___data = transform(column_name, training___data, option)
        validation_data = transform(column_name, validation_data, option)
        
    validation_data = validation_data.drop("Id", axis=1)
    training___data = training___data.drop("Id", axis=1)
    model = None
    model = RandomForestRegressor()
    model.fit(training___data, training___price)

    predictions = model.predict(validation_data)

    error = float(mean_absolute_error(validation_price, predictions))
    print(f"Error {n}: {error}")
    errors.append(error)

print(f"Mean Absolute Error of {sum(errors)/len(errors)}: {min(errors)}...{max(errors)}")

Error 0: 17389.14616438356
Error 1: 17607.13126027397
Error 2: 16445.060506849313
Error 3: 17298.92380821918
Error 4: 16806.19375342466
Error 5: 18393.497863013697
Error 6: 15254.506739726028
Error 7: 19555.07892759296
Error 8: 16872.999506849315
Error 9: 17564.224164383562
Mean Absolute Error of 17318.676269471624: 15254.506739726028...19555.07892759296


## Saved results

- Normie: Mean Absolute Error: 17787.279715068493: 16742.85882191781...19406.53876712329
- No Alley: Mean Absolute Error of 17652.627992694062: 16217.109150684932...19793.63994520548
- Basic zoning: Mean Absolute Error of 18059.097720547943: 16746.11709589041...19786.975287671234

# Generate Output

In [199]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

load_data()
for column_name, (transform, option) in transformations.items():
    full_data = transform(column_name, full_data, option)
    test_data = transform(column_name, test_data, option)
    
full_data = full_data.drop("Id", axis=1)
test_data = test_data.drop("Id", axis=1)
model = None
model = RandomForestRegressor()
model.fit(full_data, full_price)

predictions = model.predict(test_data)

submission = pd.DataFrame({'Id': test_data.index + 1461, 'SalePrice': predictions})
submission.to_csv('submission.csv', index=False)
submission

,Id,SalePrice
0,1461,124590.74
1,1462,152833.50
2,1463,174929.65
3,1464,182626.46
4,1465,200767.85
...,...,...
1454,2915,84631.61
1455,2916,85995.00
1456,2917,152046.50
1457,2918,113605.87
